In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import base64
import mimetypes
from PIL import Image
from io import BytesIO
import pandas as pd
from io import StringIO
import fitz
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'))

In [4]:
os.mkdir('images/biedronka20112023-25112023')
doc = fitz.open("images/biedronka20112023-25112023.pdf")

for idx in range(doc.page_count):
    # Select the page you want to convert (0 for the first page)
    page = doc.load_page(idx)

    # Convert it to a pixmap (an image)
    pix = page.get_pixmap()

    # Save the image
    image_path = f"images/biedronka20112023-25112023/page{idx}.png"
    pix.save(image_path)

# Close the document
doc.close()

FileExistsError: [Errno 17] File exists: 'images/biedronka20112023-25112023'

In [5]:
def chat(query, image_path):
    # Guess the MIME type of the image
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type or not mime_type.startswith('image'):
        raise ValueError("The file type is not recognized as an image")
    
    # Read the image binary data
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
        encoded_string = base64.b64encode(image_data).decode('utf-8')
    
    # Format the result with the appropriate prefix
    image_base64 = f"data:{mime_type};base64,{encoded_string}"

    # Read image for size using PIL
    image = Image.open(BytesIO(image_data))
    width, height = image.size
    print("Width:", width, "Height:", height)
    
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                    {
                        "type": "image_url",
                        "image_url": {"url": image_base64, "detail": "high"}
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content
    

In [6]:
query = ''' You are given an image of a invoice in Polish.
            I need a list of items that is in the middle of a image. There are column names there. Return the column names as a list separated by / and return the values seprated by / and a new line if there are mulitple rows.
            Return only this format and nothing else.'''

In [7]:
response = chat(query , f'images/387459570_1070736067600198_8222554501514766161_n.jpg')
response

Width: 3024 Height: 4032


'Poz./Nr art./Oznaczenie/L./Cena netto/Razem netto\n100002-PL/Windows 7 Professional/1/68,28 zł/68,28 zł'

In [17]:
offer = ''
for idx, row in enumerate(response.split('\n')):
    if idx == 0:
        columns = row.split('/')
    if '/' in row and len(row.split('/')) == 6:
        offer += row + '\n'
offer

'100002-PL/Windows 7 Professional/1/68,28 zł/68,28 zł\n'

In [22]:
# Using StringIO to simulate a file-like object
string_io = StringIO(offer.strip())

# Reading the string into a pandas DataFrame
df = pd.read_csv(string_io, sep="/", names=columns)

df

,Poz.,Nr art.,Oznaczenie,L.,Cena netto,Razem netto
0,100002-PL,Windows 7 Professional,1,"68,28 zł","68,28 zł",NaN
